In [1]:
import pandas as pd
import numpy as np

In [2]:
precinct_polling_list = pd.read_csv('precinct_polling_list.csv')
addresses = pd.read_csv('addresses.csv')
state_fips = pd.read_csv('state_fips.csv').set_index('stusps').astype(str)
display(precinct_polling_list.head())
display(addresses.head())
display(state_fips.head())

,Location Name,Street,City,State/ZIP,Country,Precinct
0,Yorkside Pizza,288 York Street,New Haven,CT 06511,USA,CON-069
1,Library,301-399 South Boulevard Drive,Bainbridge,GA 39819,USA,GEO-062
2,Luxury Boston,150-151 Tremont Street,Boston,MA 02111,USA,MAS-111
3,Ingleside Village Pizza,2395 Ingleside Avenue,Macon,GA 31204,USA,GEO-041
4,Pizza House,1007 Merchant Street,Ambridge,PA 15003,USA,MAS-018


,Street,Apt,City,State,Zip,Precinct ID
0,7 BEACON BLVD,NaN,PEABODY,MA,01960-6303,025-090
1,1847 TOWER DR,NaN,STOUGHTON,WI,53589-3539,055-015
2,4628 GREEN VALLEY RD,NaN,FAIRFIELD,CA,94534-1368,006-009
3,14 HALET ST,NaN,PORTLAND,ME,04102-1734,023-089
4,1606 BLACKBURN HEIGHTS DR,NaN,SEWICKLEY,PA,15143-8626,042-018


,stname,st
stusps,,
AL,Alabama,1
AK,Alaska,2
AZ,Arizona,4
AR,Arkansas,5
CA,California,6


In [3]:
# Fix bad precinct rows
precinct_polling_list = pd.read_csv('precinct_polling_list.csv')
bad_data = precinct_polling_list.isna().any(axis=1) # Get rows where data didnt work
display(precinct_polling_list[bad_data])

# Hearth Pizzeria
precinct_polling_list.iloc[6][4:] = precinct_polling_list.iloc[6][2:4] # shift data over
precinct_polling_list.iloc[6]['State/ZIP'] = " ".join(precinct_polling_list.iloc[6]['Street'].split()[-2:])
precinct_polling_list.iloc[6]['City'] = precinct_polling_list.iloc[6]['Street'].split()[-3]
precinct_polling_list.iloc[6]['Street'] = " ".join(precinct_polling_list.iloc[6]['Street'].split()[:-3])

# Community Center
precinct_polling_list.iloc[13][4:] = precinct_polling_list.iloc[13][3:5] # shift data over
precinct_polling_list.iloc[13]['State/ZIP'] = " ".join(precinct_polling_list.iloc[13]['City'].split()[-2:])
precinct_polling_list.iloc[13]['City'] = precinct_polling_list.iloc[13]['City'].split()[0]

# Upper Crust Pizzeria (Same fn as Hearth)
precinct_polling_list.iloc[23][4:] = precinct_polling_list.iloc[23][2:4] # shift data over
precinct_polling_list.iloc[23]['State/ZIP'] = " ".join(precinct_polling_list.iloc[23]['Street'].split()[-2:])
precinct_polling_list.iloc[23]['City'] = precinct_polling_list.iloc[23]['Street'].split()[-3]
precinct_polling_list.iloc[23]['Street'] = " ".join(precinct_polling_list.iloc[23]['Street'].split()[:-3])

# Ian's Pizza Milwaukee
precinct_polling_list.iloc[30][4:] = precinct_polling_list.iloc[30][3:5] # shift data over
precinct_polling_list.iloc[30]['State/ZIP'] = " ".join([precinct_polling_list.iloc[30]['Street'].split()[-1],
                                                        precinct_polling_list.iloc[30]['City'].split()[-1]])
precinct_polling_list.iloc[30]['City'] = precinct_polling_list.iloc[30]['Street'].split()[-2]
precinct_polling_list.iloc[30]['Street'] = " ".join(precinct_polling_list.iloc[30]['Street'].split()[:-3])

# Courthouse (extra dash)
precinct_polling_list['Precinct'] = precinct_polling_list['Precinct'].str.replace('--', '-0')

display(precinct_polling_list[bad_data])

,Location Name,Street,City,State/ZIP,Country,Precinct
6,Hearth Pizzeria,974 Great Plain Avenue Needham MA 02492,USA,MAS-006,NaN,NaN
13,Community Center,29 Godwin Avenue,Ridgewood NJ 07450,USA,NEWJ-000,NaN
23,Upper Crust Pizzeria,1782 Massachusetts Ave Cambridge MA 02140,USA,MAS-070,NaN,NaN
30,Ian's Pizza Milwaukee,2035 East North Avenue Milwaukee WI,53211,USA,WIS-067,NaN


,Location Name,Street,City,State/ZIP,Country,Precinct
6,Hearth Pizzeria,974 Great Plain Avenue,Needham,MA 02492,USA,MAS-006
13,Community Center,29 Godwin Avenue,Ridgewood,NJ 07450,USA,NEWJ-000
23,Upper Crust Pizzeria,1782 Massachusetts Ave,Cambridge,MA 02140,USA,MAS-070
30,Ian's Pizza Milwaukee,2035 East North,Milwaukee,WI 53211,USA,WIS-067


In [4]:
precinct_polling_list['State'] = precinct_polling_list['State/ZIP'].str.split(expand=True)[0]
precinct_polling_list = precinct_polling_list.join(state_fips, on='State')
precinct_polling_list['Precinct ID'] = precinct_polling_list['st'].str.zfill(3) + '-' + precinct_polling_list['Precinct'].str.split('-', expand=True)[1]
precinct_polling_list = precinct_polling_list.set_index('Precinct ID')
precinct_polling_list.head()

,Location Name,Street,City,State/ZIP,Country,Precinct,State,stname,st
Precinct ID,,,,,,,,,
009-069,Yorkside Pizza,288 York Street,New Haven,CT 06511,USA,CON-069,CT,Connecticut,9
013-062,Library,301-399 South Boulevard Drive,Bainbridge,GA 39819,USA,GEO-062,GA,Georgia,13
025-111,Luxury Boston,150-151 Tremont Street,Boston,MA 02111,USA,MAS-111,MA,Massachusetts,25
013-041,Ingleside Village Pizza,2395 Ingleside Avenue,Macon,GA 31204,USA,GEO-041,GA,Georgia,13
042-018,Pizza House,1007 Merchant Street,Ambridge,PA 15003,USA,MAS-018,PA,Pennsylvania,42


In [5]:
precinct_polling_list.sort_index()

,Location Name,Street,City,State/ZIP,Country,Precinct,State,stname,st
Precinct ID,,,,,,,,,
004-067,Organ Stop Pizza,1149 East Southern Avenue,Mesa,AZ 85204,USA,ARI-067,AZ,Arizona,4
006-000,The Pruneyard Shopping Center,1875 South Bascom Avenue,Campbell,CA 95008,USA,CAL-000,CA,California,6
006-009,Mary's Pizza Shack,3085 Jefferson Street,Napa,CA 94559,USA,CAL-009,CA,California,6
006-021,Higgins Building,108 West 2nd Street,Los Angeles,CA 90012,USA,CAL-021,CA,California,6
006-032,Senior High School,563 Carlsbad Village Drive,Carlsbad,CA 92008,USA,CAL-032,CA,California,6
006-041,Middle School Gym,443 Emerson Street,Palo Alto,CA 94301,USA,CAL-041,CA,California,6
009-069,Yorkside Pizza,288 York Street,New Haven,CT 06511,USA,CON-069,CT,Connecticut,9
012-051,Cre8tive Apparel,859 Washington Avenue,Miami Beach,FL 33139,USA,FLO-051,FL,Florida,12
012-067,Courthouse,9600-9708 West Linebaugh Avenue,Westchase,FL 33626,USA,FLO-067,FL,Florida,12


In [6]:
addresses = addresses.join(state_fips, on='State')
addresses['Precinct ID'] = addresses['st'].str.zfill(3) + '-' + addresses['Precinct ID'].str.split('-', expand=True)[1]

In [7]:
addresses

,Street,Apt,City,State,Zip,Precinct ID,stname,st
0,7 BEACON BLVD,NaN,PEABODY,MA,01960-6303,025-090,Massachusetts,25
1,1847 TOWER DR,NaN,STOUGHTON,WI,53589-3539,055-015,Wisconsin,55
2,4628 GREEN VALLEY RD,NaN,FAIRFIELD,CA,94534-1368,006-009,California,6
3,14 HALET ST,NaN,PORTLAND,ME,04102-1734,023-089,Maine,23
4,1606 BLACKBURN HEIGHTS DR,NaN,SEWICKLEY,PA,15143-8626,042-018,Pennsylvania,42
5,405 LAUREL PL,NaN,MACON,GA,31220-8761,013-041,Georgia,13
6,1011 VALENCIA AVE,NaN,CORAL GABLES,FL,33134-5536,012-051,Florida,12
7,43 BOYDEN ST,NaN,BROCKTON,MA,02302-2303,025-111,Massachusetts,25
8,1051 E STEARNS AVE,APT 36,LA HABRA,CA,90631-4857,006-021,California,6
9,10 RUSTIC DR,NaN,N BRUNSWICK,NJ,08902-4706,034-010,NewJersey,34


In [8]:
addresses = addresses.join(precinct_polling_list[['Location Name', 'Street', 'City']], on='Precinct ID', rsuffix ='_precinct')

,Street,Apt,City,State,Zip,Precinct ID,stname,st,Location Name,Street_precinct,City_precinct
0,7 BEACON BLVD,NaN,PEABODY,MA,01960-6303,025-090,Massachusetts,25,Stonewood Tavern,139 Lynnfield Street,Peabody
1,1847 TOWER DR,NaN,STOUGHTON,WI,53589-3539,055-015,Wisconsin,55,Michelangelo's Coffee House,114 State Street,Madison
2,4628 GREEN VALLEY RD,NaN,FAIRFIELD,CA,94534-1368,006-009,California,6,Mary's Pizza Shack,3085 Jefferson Street,Napa
3,14 HALET ST,NaN,PORTLAND,ME,04102-1734,023-089,Maine,23,Elementaray School Auditorium,574 Congress Street,Portland
4,1606 BLACKBURN HEIGHTS DR,NaN,SEWICKLEY,PA,15143-8626,042-018,Pennsylvania,42,Pizza House,1007 Merchant Street,Ambridge
5,405 LAUREL PL,NaN,MACON,GA,31220-8761,013-041,Georgia,13,Ingleside Village Pizza,2395 Ingleside Avenue,Macon
6,1011 VALENCIA AVE,NaN,CORAL GABLES,FL,33134-5536,012-051,Florida,12,Cre8tive Apparel,859 Washington Avenue,Miami Beach
7,43 BOYDEN ST,NaN,BROCKTON,MA,02302-2303,025-111,Massachusetts,25,Luxury Boston,150-151 Tremont Street,Boston
8,1051 E STEARNS AVE,APT 36,LA HABRA,CA,90631-4857,006-021,California,6,Higgins Building,108 West 2nd Street,Los Angeles
9,10 RUSTIC DR,NaN,N BRUNSWICK,NJ,08902-4706,034-010,NewJersey,34,Dolceria,180 Nassau Street,Princeton


# precinct.txt